In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train_df.head()

In [3]:
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_df.head()

In [4]:
train_df.shape

In [5]:
X = train_df.drop(['label'], axis=1)
y = train_df['label']

In [6]:
X.shape

In [7]:
# Normalize the data
X = X / 255.0
test_df = test_df/255.0

In [8]:
X = X.values.reshape(-1,28,28,1)
test_df = test_df.values.reshape(-1,28,28,1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.10, random_state=42)

In [10]:
X_train.shape

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(X[0][:,:,0])

In [12]:
from keras.utils import np_utils
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [25]:
y_train=tf.keras.utils.to_categorical(y_train,10)
y_val=tf.keras.utils.to_categorical(y_val,10)

In [16]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(500, activation='tanh'),
                            tf.keras.layers.Dense(200, activation='tanh'),
#                             tf.keras.layers.Dropout(0.2),  
                            tf.keras.layers.Dense(200, activation='relu'),
#                             tf.keras.layers.Dropout(0.2),                        
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

In [17]:
model.summary()

In [18]:
#model.fit(X_train,y_train,batch_size=200,nb_epoch=20,verbose=1,validation_data=(X_val,y_val))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
NUM_EPOCHS = 100

model.fit(X_train,y_train, batch_size=10000, epochs = NUM_EPOCHS, validation_data=(X_val,y_val), verbose=2)

In [20]:
y_predict = model.predict(X_val)

In [22]:
y_pred = []
for val in y_predict:
    y_pred.append(np.argmax(val))
# print(y_pred)    
#convert 0 1 to 1 and 1 0 as 0
accuracy = metrics.accuracy_score(np.argmax(y_val,axis = 1) ,y_pred)
recall = metrics.recall_score(np.argmax(y_val,axis = 1) ,y_pred,average='macro')
precision = metrics.precision_score(np.argmax(y_val,axis = 1) ,y_pred,average='macro')
f1_score = metrics.f1_score(np.argmax(y_val,axis = 1) ,y_pred,average='macro')
cm = metrics.confusion_matrix(np.argmax(y_val,axis = 1) ,y_pred)
print(cm)
print("Accuracy:",accuracy)
print("F1 score:",f1_score)
print("Recall:",recall)
print("Precision:",precision)

In [ ]:
# predict results
results = model.predict(test_df)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("digit_submission.csv",index=False)